In [ ]:
!pip install pyomo
!apt-get install -y glpk-utils

from pyomo.environ import *

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


# **PART-3**

In [ ]:
from itertools import product

total_capacity = 310
demand_medium = [20, 30, 40, 35, 15, 19, 25, 20, 18, 12, 30, 40]

# Demand multipliers
low_factor, medium_factor, high_factor = 0.7, 1.0, 1.3
demand_factors = [low_factor, medium_factor, high_factor]

probabilities = [0.3, 0.4, 0.3] #Low: 0.3, Medium: 0.4, High: 0.3)



all_scenarios = list(product(demand_factors, repeat=12))


prob_exceeding = 0 # probability of exceeding capacity

for scenario in all_scenarios:
    total_demand = sum(scenario[i] * demand_medium[i] for i in range(12))

    scenario_prob = 1
    for factor in scenario:
        scenario_prob *= probabilities[demand_factors.index(factor)]

    if total_demand > total_capacity:
        prob_exceeding += scenario_prob

print(f"Fraction of scenarios where demand exceeds capacity: {prob_exceeding:.4f}")


Fraction of scenarios where demand exceeds capacity: 0.3907


**So, 0.3907 fraction of scenarios we will have more demand than capacity**

# **PART-4**

In [ ]:
import numpy as np


plants = [1, 2, 3]
capacity = {1: 120, 2: 90, 3: 100}  # In 10,000s
cost_production = {1: 8, 2: 7, 3: 9}  # Cost per unit

distribution_centers = list(range(1, 13))
demand_medium = {1: 20, 2: 30, 3: 40, 4: 35, 5: 15, 6: 19, 7: 25, 8: 20, 9: 18, 10: 12, 11: 30, 12: 40}
prob_low, prob_medium, prob_high = 0.3, 0.4, 0.3


num_scenarios = 3000 # Generated random sampled scenarios (3000 scenarios instead of 3^12)
scenarios = []
scenario_probs = []

np.random.seed(42)
for _ in range(num_scenarios):
    scenario = {j: np.random.choice(
        [0.7 * demand_medium[j], demand_medium[j], 1.3 * demand_medium[j]],
        p=[prob_low, prob_medium, prob_high]) for j in distribution_centers}
    scenarios.append(scenario)
    scenario_probs.append(1/num_scenarios)

transport_cost = {
    (1,1):2, (1,2):3, (1,3):2, (1,4):3, (1,5):5, (1,6):4.4, (1,7):3.2, (1,8):3, (1,9):1.5, (1,10):3, (1,11):6.0, (1,12):1.0,
    (2,1):1.9, (2,2):3, (2,3):2.4, (2,4):2.4, (2,5):2, (2,6):4.1, (2,7):2.8, (2,8):3, (2,9):1.7, (2,10):2, (2,11):1.0, (2,12):4.5,
    (3,1):3.1, (3,2):1.8, (3,3):2.8, (3,4):3.4, (3,5):2, (3,6):2.9, (3,7):2.2, (3,8):3.2, (3,9):2, (3,10):2.8, (3,11):4.5, (3,12):6.0
}

selling_price = 20
scrap_price = 5
penalty_cost = 7


model = ConcreteModel()

# Decision variables
model.y = Var(plants, within=NonNegativeReals)  # Production at each plant
model.x = Var(plants, distribution_centers, range(num_scenarios), within=NonNegativeReals)  # Transportation
model.u = Var(plants, range(num_scenarios), within=NonNegativeReals)  # Unsold bread
model.v = Var(distribution_centers, range(num_scenarios), within=NonNegativeReals)  # Lost sales

# Objective function
def expected_profit_rule(model):
    return sum(
        scenario_probs[s] * (
            sum((selling_price - transport_cost[i, j]) * model.x[i, j, s] for i in plants for j in distribution_centers)
            - sum(cost_production[i] * model.y[i] for i in plants)
            - sum(scrap_price * model.u[i, s] for i in plants)
            - sum(penalty_cost * model.v[j, s] for j in distribution_centers)
        ) for s in range(num_scenarios)
    )

model.obj = Objective(rule=expected_profit_rule, sense=maximize)

# Constraints
# Production limits
def production_limit_rule(model, i):
    return model.y[i] <= capacity[i]
model.production_limit = Constraint(plants, rule=production_limit_rule)

# Flow conservation
def flow_balance_rule(model, i, s):
    return sum(model.x[i, j, s] for j in distribution_centers) + model.u[i, s] == model.y[i]
model.flow_balance = Constraint(plants, range(num_scenarios), rule=flow_balance_rule)

# Demand fulfillment
def demand_fulfillment_rule(model, j, s):
    return sum(model.x[i, j, s] for i in plants) + model.v[j, s] == scenarios[s][j]
model.demand_fulfillment = Constraint(distribution_centers, range(num_scenarios), rule=demand_fulfillment_rule)



solver = SolverFactory('glpk',executable='/usr/bin/glpsol')
result = solver.solve(model, tee=True)


optimal_production = {i: model.y[i].value for i in plants}
expected_profit = model.obj()

print("Optimal Production Quantities:", optimal_production)
print("Expected Profit:", expected_profit)


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp0fkvk_6m.glpk.raw --wglp /tmp/tmpoikn0sms.glpk.glp --cpxlp
 /tmp/tmpc3zbw13s.pyomo.lp
Reading problem data from '/tmp/tmpc3zbw13s.pyomo.lp'...
45003 rows, 153003 columns, 270003 non-zeros
711027 lines were read
Writing problem data to '/tmp/tmpoikn0sms.glpk.glp'...
657018 lines were written
GLPK Simplex Optimizer 5.0
45003 rows, 153003 columns, 270003 non-zeros
Preprocessing...
45000 rows, 108003 columns, 225000 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 45000
*     0: obj =  -2.126216400e+03 inf =   0.000e+00 (108000)
Perturbing LP to avoid stalling [199]...
*  2021: obj =  -2.104872400e+03 inf =   0.000e+00 (92395) 7
*  4532: obj =  -1.559979800e+03 inf =   0.000e+00 (88414) 12
*  6965: obj =  -1.389159007e+03 inf =   8.900e-08 (73052) 8
*  8674: obj =  -1

Optimal Production Quantities (in 10,000's):

PLANT-1 : 120

PLANT-2 : 90

PLANT-3: 94.9

(We are taking 3000 samples out of 3^12 samples)

EXPECTED PROFIT: 2809.588023333373  

# **PART-5**

**QUANTITIES PRODUCED AT ** (In 10,000's)

**PLANT 1:**

Deterministic LP: 33

Stochastic LP: 120

**PLANT 2:**

Deterministic LP: 26

Stochastic LP: 90

**PLANT 3:**

Deterministic LP: 28

Stochastic LP: 94.9
